In [4]:
import findspark
findspark.init()

#Import PySpark libraries
import pyspark
import random
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType,StringType,LongType,ArrayType
from pyspark.sql.functions import pandas_udf, PandasUDFType, udf, approx_count_distinct
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col, max as _max, lit
from pyspark.sql import functions as F
import json

In [5]:
#Connect to Spark Cluster
conf = SparkConf().setAppName("ColumnTypes").setMaster("spark://192.168.56.1:7077")
sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)

#Print Spark Version
print("Spark Version: " + sc.version)
print("PySpark Version: " + pyspark.__version__)

Spark Version: 2.3.2
PySpark Version: 2.3.2


<h3>Functions</h3>

In [46]:
def _renameDF_Columns(df):
    ORIGINAL_COLUMN_NAMES = df.columns
    NEW_COLUMN_NAMES = [str(x).upper().strip() for x in ORIGINAL_COLUMN_NAMES]                

    for orig, new in zip(ORIGINAL_COLUMN_NAMES,NEW_COLUMN_NAMES):
        df=df.withColumnRenamed(orig,new)

    return df

import re
import datetime
import pandas as pd
def intTryParse(value):
    value = str(value)
    res = re.search('e', str(value), re.IGNORECASE)
    if not res:
        try:
            int(value)
            return True

        except Exception:
            return isFloatWholeNumber(value)
    else:
        return False

def floatTryParse(value):
    value = str(value)
    res = re.search('e', str(value), re.IGNORECASE)
    if not res:
         try:
            float(value)
            return True
         except ValueError:
            return False
    else:
        return False
    
def isFloatWholeNumber(x):
    value = str(x)
    try:
        fval = float(value)
        return fval.is_integer()

    except Exception:
        return False
    
    
def dateTryParse(s):
    s = str(s)
    isDate = False

    formats = ["%m/%d/%Y", "%Y-%m-%d", "%Y-%m-%d %H:%M:%S"]
    for f in formats:
        try:
            d = datetime.datetime.strptime(s, f)
            return True,d
        except ValueError:
            pass

    return isDate, None


def isNull(val):
    if val==None:
        return True
    if pd.isnull(val):
        return True
    if str(val)=="nan":
        return True
    return False

#returns column datatype.. Float, Int, Date or String
def getType(x):
    isContainsPeriod = False
    if str(x).find(".") != -1:
        isContainsPeriod = True

    if intTryParse(x) and not (str(x).startswith('0') and not isContainsPeriod and not (str(x) == '0')):
        return "int"

    if floatTryParse(x) and not (str(x).startswith('0') and not isContainsPeriod):
        return "float"

    isDate, dtConverted =  dateTryParse(str(x))
    if isDate:
        return "date"

    return "string"

In [74]:
@pandas_udf(returnType=ArrayType(StringType()))
def getType_pudf(ser):
    converted = [getType(v) for v in ser]
    list_set = set(converted) 
    ltypes = (list(list_set)) 

    #Determine Length
    if "string" in ltypes:
        lengths = [len(v) for v in ser]
        maxLength = max(lengths)
    
    elif ("string" not in ltypes) and ("date" not in ltypes):
        maxLength = max(ser)
    
    else:
        maxLength = 0
    
    isNullable = True
    
    #Determine Nullable
    convNull = [isNull(v) for v in ser]
    list_set = set(convNull) 
    nullTypes = (list(list_set)) 
    
    d = {}
    d["types"] = ltypes  
    d["length"] = maxLength
    d["nullTypes"] = nullTypes

    return json.dumps(d)


def array_to_string_original(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

def array_to_string(my_list):
    #return ','.join([str(elem) for elem in my_list])
    return str(my_list)

array_to_string_udf = udf(array_to_string, returnType=StringType())

In [75]:
#Read DataSet
df = sqlContext.read.csv("1500000SalesRecords.csv",header=True, sep=",", inferSchema=False)
#Strip Column Names and UpperCase all Columns
df = _renameDF_Columns(df)
#Print Columns
df.head()

Row(REGION='Sub-Saharan Africa', COUNTRY='South Africa', ITEM TYPE='Fruits', SALES CHANNEL='Offline', ORDER PRIORITY='M', ORDER DATE='7/27/2012', ORDER ID='443368995', SHIP DATE='7/28/2012', UNITS SOLD='1593', UNIT PRICE='9.33', UNIT COST='6.92', TOTAL REVENUE='14862.69', TOTAL COST='11023.56', TOTAL PROFIT='3839.13')

In [76]:
#GET UNIQUE VALUES TO ARRAY IN A DATAFRAME..
COLUMN_NAMES = df.columns

dfU = df.withColumn("Temp",lit("a"))
exprs = [F.collect_set(colName) for colName in COLUMN_NAMES]
dfU = dfU.groupby('Temp').agg(*exprs)
dfU = dfU.drop("Temp")
dfU = dfU.toDF(*COLUMN_NAMES)
dfU.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              REGION|             COUNTRY|           ITEM TYPE|    SALES CHANNEL|ORDER PRIORITY|          ORDER DATE|            ORDER ID|           SHIP DATE|          UNITS SOLD|          UNIT PRICE|           UNIT COST|       TOTAL REVENUE|          TOTAL COST|        TOTAL PROFIT|
+--------------------+--------------------+--------------------+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Middle East and ...|[Italy, Slovakia,...|[Beverages, Perso...|[Online, Offline]|  [M, C, H, L]|[7/20/2016, 2/6/2...|[863378036, 44360...|

In [77]:
#apply UDF to grouped by data
dfTypes = dfU.select(
    *[getType_pudf(col(col_name)).name(col_name) for col_name in dfU.columns]
)

TypeError: 'str' object is not callable

In [ ]:
#convert to string to allow printing
dfTypes_cln = dfTypes.select(
    *[array_to_string_udf(col(col_name)).name(col_name) for col_name in dfTypes.columns]
)

In [ ]:
dfPandas = dfTypes_cln.toPandas()

In [ ]:
dfPandas

In [ ]:
d = dfPandas.to_dict()
d

In [ ]:
d

In [ ]:
v = d['TOTAL REVENUE'][0]
print(v)
jo = json.loads(v)
jo['types']

In [ ]:
for col in d:
    jo = json.loads(d[col][0])
    
    print(col + "   " + str(jo["types"]) + "   "  + str(jo["length"]))

In [ ]:
dfTypes_cln.write.csv('type_df.csv',header=True)

In [ ]:
#Convert to Dictionary
new_dict = dfTypes.toPandas().to_dict(orient='list')
new_dict